In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install -q groq pandas tqdm sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 76.0 MB/s eta 0:00:00


In [3]:
import os
import json
import time
import faiss
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from sentence_transformers import SentenceTransformer
from groq import Groq

In [4]:
OUT_DIR = "/content/drive/MyDrive/rag"
EMBED_MODEL = "BAAI/bge-m3"
CSV_OUT = f"{OUT_DIR}/rag_finetune_dataset.csv"
GEN_MODEL = "llama-3.1-8b-instant"
PROMPT_TEMPLATE = """Bạn đang tạo dữ liệu huấn luyện cho hệ thống Retrieval-Augmented Generation (RAG).

Dựa trên NỘI DUNG bên dưới:
- Viết 7-8 câu hỏi thực tế và đa dạng mà người dùng có thể hỏi, nếu nội dung chung chung hoặc không mang thông tin hỗ trợ học sinh thì có thể chỉ cần 3-4 câu hỏi
- Tập câu hỏi nên bao gồm cả mức độ khái quát và chi tiết
- Mỗi câu hỏi PHẢI trả lời được CHỈ dựa vào nội dung được cung cấp
- KHÔNG được suy đoán hay bịa thêm thông tin
- KHÔNG viết câu trả lời
- KHÔNG giải thích
- KHÔNG thêm văn bản ngoài JSON

Trả về DUY NHẤT JSON hợp lệ theo đúng định dạng sau:

[
  {{
    "question": "..."
  }}
]

NỘI DUNG:
\"\"\"
{context}
\"\"\"
"""

In [ ]:
from google.colab import userdata

client = Groq(api_key=userdata.get('groqApiKey'))

index = faiss.read_index(f"{OUT_DIR}/rag.index")

with open(f"{OUT_DIR}/metadata.json", "r", encoding="utf-8") as f:
    docs = json.load(f)

print("Index size:", index.ntotal)
print("Docs count:", len(docs))
assert index.ntotal == len(docs)

docs_by_source = defaultdict(list)
for i, d in enumerate(docs):
    docs_by_source[d["metadata"]["source"]].append((i, d))

def generate(prompt):
    completion = client.chat.completions.create(
        model=GEN_MODEL,
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.2,
        max_tokens=1024,
    )
    return completion.choices[0].message.content

def build_context(chunks, idx, window=1):
    parts = []
    for j in range(max(0, idx-window), min(len(chunks), idx+window+1)):
        parts.append(chunks[j][1]["text"])
    return "\n\n".join(parts)

rows = []

for source, chunks in tqdm(docs_by_source.items(), desc="Processing documents"):
    for idx, (global_idx, doc) in tqdm(
        enumerate(chunks),
        total=len(chunks),
        desc=source,
        position=1,
        leave=False

    ):
        context = build_context(chunks, idx, window=1)

        if len(doc['text']) < 150:
            continue

        prompt = PROMPT_TEMPLATE.format(context=context[:6000])

        try:
            text = generate(prompt).strip()
            qa_pairs = json.loads(text)

            for qa in qa_pairs:
                rows.append({
                    "source": source,
                    "chunk_idx": idx,
                    "section": doc["metadata"]["section"],
                    "type": doc["metadata"]["type"],
                    "context": context,
                    "question": qa["question"],
                })

        except Exception as e:
            print(f"[SKIP] {source} chunk {idx}: {e}")
            time.sleep(3)
            continue

Index size: 681
Docs count: 681


Processing documents:   0%|          | 0/31 [00:00<?, ?it/s]
01_1%202015%20TT%20Lien%20tich_QD%20danh%20gia%20QP-AN.pdf:   0%|          | 0/15 [00:00<?, ?it/s]
01_1%202015%20TT%20Lien%20tich_QD%20danh%20gia%20QP-AN.pdf:   7%|▋         | 1/15 [00:00<00:10,  1.33it/s]
01_1%202015%20TT%20Lien%20tich_QD%20danh%20gia%20QP-AN.pdf:  13%|█▎        | 2/15 [00:01<00:07,  1.64it/s]
01_1%202015%20TT%20Lien%20tich_QD%20danh%20gia%20QP-AN.pdf:  20%|██        | 3/15 [00:02<00:08,  1.39it/s]
01_1%202015%20TT%20Lien%20tich_QD%20danh%20gia%20QP-AN.pdf:  27%|██▋       | 4/15 [00:02<00:07,  1.41it/s]
01_1%202015%20TT%20Lien%20tich_QD%20danh%20gia%20QP-AN.pdf:  33%|███▎      | 5/15 [00:03<00:08,  1.23it/s]
01_1%202015%20TT%20Lien%20tich_QD%20danh%20gia%20QP-AN.pdf:  40%|████      | 6/15 [00:04<00:07,  1.24it/s]
01_1%202015%20TT%20Lien%20tich_QD%20danh%20gia%20QP-AN.pdf:  47%|████▋     | 7/15 [00:07<00:11,  1.45s/it]
01_1%202015%20TT%20Lien%20tich_QD%20danh%20gia%20QP-AN.pdf:  53%|█████▎    | 8/15 [00:16<00

[SKIP] 03_1%20HUST's%20ACADEMIC%20REGULATIONS-2025_Final.pdf chunk 1: Expecting ',' delimiter: line 21 column 62 (char 934)



03_1%20HUST's%20ACADEMIC%20REGULATIONS-2025_Final.pdf:   2%|▏         | 2/98 [00:19<16:17, 10.18s/it]
03_1%20HUST's%20ACADEMIC%20REGULATIONS-2025_Final.pdf:   3%|▎         | 3/98 [00:23<12:04,  7.63s/it]
03_1%20HUST's%20ACADEMIC%20REGULATIONS-2025_Final.pdf:   4%|▍         | 4/98 [00:31<12:08,  7.75s/it]
03_1%20HUST's%20ACADEMIC%20REGULATIONS-2025_Final.pdf:   5%|▌         | 5/98 [00:38<11:24,  7.36s/it]
03_1%20HUST's%20ACADEMIC%20REGULATIONS-2025_Final.pdf:   7%|▋         | 7/98 [00:45<08:01,  5.29s/it]
03_1%20HUST's%20ACADEMIC%20REGULATIONS-2025_Final.pdf:   8%|▊         | 8/98 [00:53<09:20,  6.23s/it]
03_1%20HUST's%20ACADEMIC%20REGULATIONS-2025_Final.pdf:   9%|▉         | 9/98 [01:01<09:56,  6.70s/it]
03_1%20HUST's%20ACADEMIC%20REGULATIONS-2025_Final.pdf:  10%|█         | 10/98 [01:09<10:17,  7.02s/it]
03_1%20HUST's%20ACADEMIC%20REGULATIONS-2025_Final.pdf:  11%|█         | 11/98 [01:18<10:53,  7.51s/it]
03_1%20HUST's%20ACADEMIC%20REGULATIONS-2025_Final.pdf:  12%|█▏        | 12/98 [

[SKIP] 03_1%20HUST's%20ACADEMIC%20REGULATIONS-2025_Final.pdf chunk 42: Expecting ',' delimiter: line 15 column 106 (char 716)


...

[SKIP] QD_ngoai_ngu_tu_K68_CQ_final.pdf chunk 43: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01kdynzv4kfchrsqb4ca8yym5x` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499541, Requested 936. Please try again in 1m22.4256s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}



QD_ngoai_ngu_tu_K68_CQ_final.pdf:  98%|█████████▊| 46/47 [03:37<00:08,  8.47s/it]

[SKIP] QD_ngoai_ngu_tu_K68_CQ_final.pdf chunk 46: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01kdynzv4kfchrsqb4ca8yym5x` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500000, Requested 655. Please try again in 1m53.184s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}



Processing documents: 100%|██████████| 31/31 [1:19:39<00:00, 154.19s/it]


Saved CSV: /content/drive/MyDrive/rag/rag_finetune_dataset.csv
Total QA pairs: 2477


In [ ]:
df = pd.DataFrame(rows)
df.to_csv(CSV_OUT, index=False, encoding="utf-8")

print("Saved CSV:", CSV_OUT)
print("Total QA pairs:", len(df))

Saved CSV: /content/drive/MyDrive/rag/rag_finetune_dataset.csv
Total QA pairs: 2477


In [ ]:
client = Groq(api_key=os.environ["API_KEY"])
index = faiss.read_index(f"{OUT_DIR}/rag.index")

with open(f"{OUT_DIR}/metadata.json", "r", encoding="utf-8") as f:
    docs = json.load(f)

print("Index size:", index.ntotal)
print("Docs count:", len(docs))
assert index.ntotal == len(docs)

docs_by_source = defaultdict(list)
for i, d in enumerate(docs):
    docs_by_source[d["metadata"]["source"]].append((i, d))

processed_chunks = set()

if os.path.exists(CSV_OUT):
    existing_df = pd.read_csv(CSV_OUT)

    processed_chunks = set(
        existing_df.apply(
            lambda r: f"{r['source']}::{r['chunk_idx']}",
            axis=1
        )
    )

print("Already processed chunks:", len(processed_chunks))

def generate(prompt: str) -> str:
    completion = client.chat.completions.create(
        model=GEN_MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        max_tokens=1024,
    )
    return completion.choices[0].message.content


def build_context(chunks, idx, window=1):
    parts = []
    for j in range(max(0, idx - window), min(len(chunks), idx + window + 1)):
        parts.append(chunks[j][1]["text"])
    return "\n\n".join(parts)

rows = []
failed = []

for source, chunks in tqdm(docs_by_source.items(), desc="Processing documents"):
    for idx, (global_idx, doc) in tqdm(
        enumerate(chunks),
        total=len(chunks),
        desc=source,
        position=1,
        leave=False
    ):
        chunk_key = f"{source}::{idx}"

        if chunk_key in processed_chunks:
            continue

        if len(doc["text"]) < 150:
            continue

        context = build_context(chunks, idx, window=1)
        prompt = PROMPT_TEMPLATE.format(context=context[:6000])

        try:
            text = generate(prompt).strip()
            qa_pairs = json.loads(text)

            for qa in qa_pairs:
                rows.append({
                    "source": source,
                    "chunk_idx": idx,
                    "section": doc["metadata"]["section"],
                    "type": doc["metadata"]["type"],
                    "context": context,
                    "question": qa["question"],
                })

        except Exception as e:
            print(f"[SKIP] {source} chunk {idx}: {e}")
            failed.append({
                "source": source,
                "chunk_idx": idx,
                "error": str(e)
            })
            time.sleep(3)
            continue

if rows:
    new_df = pd.DataFrame(rows)

    if os.path.exists(CSV_OUT):
        new_df.to_csv(CSV_OUT, mode="a", header=False, index=False, encoding="utf-8")
    else:
        new_df.to_csv(CSV_OUT, index=False, encoding="utf-8")

    print("New QA pairs added:", len(new_df))
else:
    print("No new QA pairs generated.")

if failed:
    failed_df = pd.DataFrame(failed)
    failed_df.to_csv("failed_chunks.csv", index=False, encoding="utf-8")
    print("Failed chunks saved:", len(failed))


Index size: 681
Docs count: 681
Already processed chunks: 303


Processing documents:   0%|          | 0/31 [00:00<?, ?it/s]
01_1%202015%20TT%20Lien%20tich_QD%20danh%20gia%20QP-AN.pdf:   0%|          | 0/15 [00:00<?, ?it/s]
                                                                                                  
01_3%20HD%20hoc%20chuyen%20tiep%20ky%20su%20180%20TC_Final.pdf:   0%|          | 0/4 [00:00<?, ?it/s]
                                                                                                     
02_%20Th%E1%BB%A7%20th%E1%BB%A5c_Bi%E1%BB%83u%20m%E1%BA%ABu%20(k%C3%A8m%20Quy%20ch%E1%BA%BF%20%C4%90T).pdf:   0%|          | 0/9 [00:00<?, ?it/s]
                                                                                                                                                 
02_gioi%20thieu%20NoteBookLM.pdf:   0%|          | 0/24 [00:00<?, ?it/s]
                                                                        
03_1%20HUST's%20ACADEMIC%20REGULATIONS-2025_Final.pdf:   0%|          | 0/98 [00:00<?, ?it/s]

[SKIP] 03_1%20HUST's%20ACADEMIC%20REGULATIONS-2025_Final.pdf chunk 1: Expecting ',' delimiter: line 24 column 60 (char 736)



03_1%20HUST's%20ACADEMIC%20REGULATIONS-2025_Final.pdf:   2%|▏         | 2/98 [00:03<03:04,  1.92s/it]
03_1%20HUST's%20ACADEMIC%20REGULATIONS-2025_Final.pdf:  44%|████▍     | 43/98 [00:05<00:05, 10.64it/s]

[SKIP] 03_1%20HUST's%20ACADEMIC%20REGULATIONS-2025_Final.pdf chunk 58: Unterminated string starting at: line 36 column 17 (char 3749)


...

[SKIP] QD%20NN%20DHCQ-2020-2021-1501.pdf chunk 49: Unterminated string starting at: line 114 column 17 (char 3796)



Processing documents:  87%|████████▋ | 27/31 [42:16<24:44, 371.11s/it]
QD%20ban%20hanh%20QD%20chuyen%20doi%20hoc%20phan%20tuong%20duong.pdf:   0%|          | 0/14 [00:00<?, ?it/s]
QD%20ban%20hanh%20QD%20chuyen%20doi%20hoc%20phan%20tuong%20duong.pdf:   7%|▋         | 1/14 [00:09<02:09,  9.97s/it]
QD%20ban%20hanh%20QD%20chuyen%20doi%20hoc%20phan%20tuong%20duong.pdf:  14%|█▍        | 2/14 [00:22<02:14, 11.22s/it]
QD%20ban%20hanh%20QD%20chuyen%20doi%20hoc%20phan%20tuong%20duong.pdf:  21%|██▏       | 3/14 [00:33<02:05, 11.41s/it]
QD%20ban%20hanh%20QD%20chuyen%20doi%20hoc%20phan%20tuong%20duong.pdf:  29%|██▊       | 4/14 [00:46<02:00, 12.04s/it]
QD%20ban%20hanh%20QD%20chuyen%20doi%20hoc%20phan%20tuong%20duong.pdf:  36%|███▌      | 5/14 [00:59<01:52, 12.45s/it]
QD%20ban%20hanh%20QD%20chuyen%20doi%20hoc%20phan%20tuong%20duong.pdf:  43%|████▎     | 6/14 [01:12<01:38, 12.37s/it]
QD%20ban%20hanh%20QD%20chuyen%20doi%20hoc%20phan%20tuong%20duong.pdf:  50%|█████     | 7/14 [01:24<01:25, 12.22s/it]


[SKIP] QD%20ban%20hanh%20QD%20to%20chuc%20day%20hoc%20tren%20nen%20tang%20CN%20ket%20noi%20-%20truc%20tuyen.pdf chunk 12: Expecting ',' delimiter: line 24 column 304 (char 1837)



QD%20ban%20hanh%20QD%20to%20chuc%20day%20hoc%20tren%20nen%20tang%20CN%20ket%20noi%20-%20truc%20tuyen.pdf:  65%|██████▌   | 13/20 [02:42<01:33, 13.31s/it]
QD%20ban%20hanh%20QD%20to%20chuc%20day%20hoc%20tren%20nen%20tang%20CN%20ket%20noi%20-%20truc%20tuyen.pdf:  70%|███████   | 14/20 [02:52<01:13, 12.29s/it]
QD%20ban%20hanh%20QD%20to%20chuc%20day%20hoc%20tren%20nen%20tang%20CN%20ket%20noi%20-%20truc%20tuyen.pdf:  75%|███████▌  | 15/20 [03:05<01:02, 12.43s/it]
QD%20ban%20hanh%20QD%20to%20chuc%20day%20hoc%20tren%20nen%20tang%20CN%20ket%20noi%20-%20truc%20tuyen.pdf:  80%|████████  | 16/20 [03:17<00:49, 12.41s/it]
QD%20ban%20hanh%20QD%20to%20chuc%20day%20hoc%20tren%20nen%20tang%20CN%20ket%20noi%20-%20truc%20tuyen.pdf:  85%|████████▌ | 17/20 [03:29<00:36, 12.19s/it]
QD%20ban%20hanh%20QD%20to%20chuc%20day%20hoc%20tren%20nen%20tang%20CN%20ket%20noi%20-%20truc%20tuyen.pdf:  90%|█████████ | 18/20 [03:41<00:24, 12.22s/it]
QD%20ban%20hanh%20QD%20to%20chuc%20day%20hoc%20tren%20nen%20tang%20CN%20ket

New QA pairs added: 2709
Failed chunks saved: 13


In [ ]:
from collections import defaultdict
import csv

groups = defaultdict(list)

with open(CSV_OUT, "r", encoding="utf-8") as f:
    for row in csv.DictReader(f):
        groups[row["context"]].append(row["question"])

for context, questions in groups.items():
    print("CONTEXT:")
    print(context)
    print('\n', "-" * 200)
    print("QUESTIONS:")
    for q in questions:
        print("-", q)
    print('\n', "-" * 200, '\n', "-" * 200, '\n\n')

CONTEXT:
<unknown>

BỘ GIÁO DỤC VÀ ĐÀO TẠO BỘ LAO ĐỘNG- THƯƠNG BINH VÀ XÃ HỘI

Số: 18/2015/TTLT-BGD Đ T-

CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập- Tự do- Hạnh phúc

Hà Nội, ngày 08 tháng 9 năm 2015

BLĐTBXH

Quy định tổ chức dạy, học và đánh giá kết quả học tập môn học Giáo dục quốc phòng và an ninh trong các cơ sở giáo dục nghề nghiệp, cơ sở giáo dục đại học

Căn cứ Luật Giáo dục ngày 14 tháng 6 năm 2005; Luật sửa đổi, bổ sung một số điều của Luật Giáo dục ngày 25 tháng 11 năm 2009;

Căn cứ Luật Giáo dục đại học ngày 18 tháng 6 n ă

m 2012;

Căn cứ Luật Giáo dục quốc phòng và an ninh ngày 19 tháng 6 n ă

m 2013;

Căn cứ Luật Giáo dục nghề nghiệp ngày 27 tháng 11 n ă

m 2014;

Căn cứ Nghị định số 32/2008/N Đ -CP ngày 19 tháng 3 năm 2008 của Chính phủ quy định chức năng, nhiệm vụ, quyền hạn và cơ cấu tổ chức của Bộ Giáo dục và Đào tạo;

Căn cứ Nghị định số 106/2012/N Đ -CP ngày 20 tháng 12 năm 2012 của Chính phủ quy định chức năng, nhiệm vụ, quyền hạn và cơ cấu tổ chức của Bộ Lao độn